In [31]:
# Check CUDA version
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [32]:
# Install CUDA package
!pip install git+https://github.com/afnan47/cuda.git

  Cloning https://github.com/afnan47/cuda.git to /tmp/pip-req-build-aa7mu_o7
  Running command git clone --filter=blob:none --quiet https://github.com/afnan47/cuda.git /tmp/pip-req-build-aa7mu_o7
  Resolved https://github.com/afnan47/cuda.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done


In [33]:
# Load nvcc plugin
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [34]:
%%writefile matrix_mult.cu
#include<stdio.h>
#include<cuda_runtime.h>

__global__ void matproduct(int *l, int *m, int *n, int row1, int col1, int row2, int col2)
{
    int x = blockIdx.x;
    int y = blockIdx.y;
    int k;

    n[col2*y+x] = 0;
    for(k = 0; k < col1; k++)
    {
        n[col2*y+x] += l[col1*y+k] * m[col2*k+x];
    }
}

int main()
{
    int row1, row2, col1, col2;

    printf("Enter the number of rows in matrix 1: ");
    scanf("%d", &row1);

    printf("Enter the number of columns in matrix 1: ");
    scanf("%d", &col1);

    printf("Enter the number of rows in matrix 2: ");
    scanf("%d", &row2);

    printf("Enter the number of columns in matrix 2: ");
    scanf("%d", &col2);

    printf("\n");

    int a[row1][col1];
    int b[row2][col2];
    int c[row1][col2];
    int *d, *e, *f;
    int i, j;

    printf("\nEnter elements of first matrix:\n");
    for(i = 0; i < row1; i++)
    {
        for(j = 0; j < col1; j++)
        {
            scanf("%d", &a[i][j]);
        }
    }

    printf("\nEnter elements of second matrix:\n");
    for(i = 0; i < row2; i++)
    {
        for(j = 0; j < col2; j++)
        {
            scanf("%d", &b[i][j]);
        }
    }

    cudaMalloc((void **)&d, row1 * col1 * sizeof(int));
    cudaMalloc((void **)&e, row2 * col2 * sizeof(int));
    cudaMalloc((void **)&f, row1 * col2 * sizeof(int));

    cudaMemcpy(d, a, row1 * col1 * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(e, b, row2 * col2 * sizeof(int), cudaMemcpyHostToDevice);

    dim3 grid(col2, row1);

    matproduct<<<grid, 1>>>(d, e, f, row1, col1, row2, col2);

    cudaMemcpy(c, f, row1 * col2 * sizeof(int), cudaMemcpyDeviceToHost);

    printf("\nProduct of two matrices:\n");
    for(i = 0; i < row1; i++)
    {
        for(j = 0; j < col2; j++)
        {
            printf("%d\t", c[i][j]);
        }
        printf("\n");
    }

    cudaFree(d);
    cudaFree(e);
    cudaFree(f);

    return 0;
}


Overwriting matrix_mult.cu


In [35]:
!nvcc matrix_mult.cu -o matrix_mult
!./matrix_mult

Enter the number of rows in matrix 1: 2
Enter the number of columns in matrix 1: 2
Enter the number of rows in matrix 2: 2
Enter the number of columns in matrix 2: 2


Enter elements of first matrix:
1
2
3
4

Enter elements of second matrix:
5
4
1
2

Product of two matrices:
7	8	
19	20	
